In [1]:
import os
import numpy as np
import pandas as pd
import xgboost

import optuna
from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, root_mean_squared_error 
import joblib

logical_cores = os.cpu_count()
print(f"Number of logical CPU cores: {logical_cores}")

num_workers = max(1, logical_cores // 2)
print(f"Number of workers set to: {num_workers}")

def is_gpu_available():
    try:
        import torch
        return torch.cuda.is_available()
    except ImportError:
        return False

gpu_available = is_gpu_available()
print(f"GPU available: {gpu_available}")

print(xgboost.build_info())

Number of logical CPU cores: 16
Number of workers set to: 8
GPU available: True
{'BUILTIN_PREFETCH_PRESENT': False, 'CUDA_VERSION': [12, 4], 'DEBUG': False, 'MM_PREFETCH_PRESENT': True, 'THRUST_VERSION': [2, 3, 2], 'USE_CUDA': True, 'USE_DLOPEN_NCCL': False, 'USE_FEDERATED': False, 'USE_NCCL': False, 'USE_OPENMP': True, 'USE_RMM': False, 'libxgboost': 'C:\\Users\\ng_mi\\Anaconda\\envs\\pytorch-gpu\\Lib\\site-packages\\xgboost\\lib\\xgboost.dll'}


In [2]:
def load_or_create_ticker_df(csv_file_path):
    """
    Load the existing ticker DataFrame from a CSV file if it exists,
    otherwise create a new DataFrame with predefined column types.
    Ensure the DataFrame has the specified columns, add any missing columns,
    and rearrange the columns in alphabetical order, excluding 'Ticker_Symbol'.

    Args:
    csv_file_path (str): The path to the CSV file.

    Returns:
    pd.DataFrame: The loaded or newly created DataFrame.
    """
    # Define the column types
    column_types = {
        "Ticker_Symbol": str,
        "Best_Cov1D_Classification_Accuracy": float,
        "Best_Cov1D_Classification_Path": str,
        "Best_Cov1D_Regression_RMSE": float,
        "Best_Cov1D_Regression_Path": str,
        "Best_LSTM_Classification_Accuracy": float,
        "Best_LSTM_Classification_Path": str,
        "Best_LSTM_Regression_RMSE": float,
        "Best_LSTM_Regression_Path": str,
        "Best_Transformer_Classification_Accuracy": float,
        "Best_TransformerClassification_Path": str,
        "Best_Transformer_Regression_RMSE": float,
        "Best_Transformer_Regression_Path": str,
        "Best_XGBClassifier_Classification_Accuracy": float,
        "Best_XGBClassifier_Classification_Path": str,
        "Best_XGBRegressor_Regression_RMSE": float,
        "Best_XGBRegressor_Regression_Path": str
    }


    if os.path.isfile(csv_file_path):
        # Load the existing file into a DataFrame
        ticker_df = pd.read_csv(csv_file_path)
        
        # Ensure all specified columns are present
        for column, dtype in column_types.items():
            if column not in ticker_df.columns:
                ticker_df[column] = pd.Series(dtype=dtype)
        
        # Reorder columns alphabetically, excluding 'Ticker_Symbol'
        columns = ["Ticker_Symbol"] + sorted([col for col in ticker_df.columns if col != "Ticker_Symbol"])
        ticker_df = ticker_df[columns]
    else:
        # Create a new DataFrame with the specified column types
        ticker_df = pd.DataFrame(columns=column_types.keys()).astype(column_types)
    
    return ticker_df

csv_file_path = "../ticker-best-model.csv"
ticker_df = load_or_create_ticker_df(csv_file_path)

In [3]:
os.makedirs('../feature-importances/xbclassifier', exist_ok=True)
os.makedirs('../feature-importances/xbregressor', exist_ok=True)
os.makedirs('../models/xgboost/xbclassifier', exist_ok=True)
os.makedirs('../models/xgboost/xbregressor', exist_ok=True)

path = '../data/train'

ticker_list = []

if os.path.exists(path):
    ticker_list = [os.path.splitext(f)[0] for f in os.listdir(path) if f.endswith('.csv')]


In [4]:
def preprocess_data(df):
    if df.isna().sum().sum() > 0 or df.isin([float('inf'), float('-inf')]).sum().sum() > 0:
        df = df.replace([float('inf'), float('-inf')], float('nan')).dropna()

    df = df.dropna()

    columns_to_drop = [
        'NEXT_DAY_CLOSEPRICE', 'DAILY_CLOSEPRICE_CHANGE', 'CLOSEPRICE_DIRECTION',
        'DAILY_MIDPRICE', 'NEXT_DAY_MIDPRICE', 'DAILY_MIDPRICE_CHANGE', 'MIDPRICE_DIRECTION', 'Date'
    ]
    X = df.drop(columns=columns_to_drop)
    y_classifier = (df['DAILY_CLOSEPRICE_CHANGE'] > 0).astype(int)
    y_regressor = df['DAILY_CLOSEPRICE_CHANGE']

    return X, y_classifier, y_regressor

In [5]:
def xbclassifier_hyperparameters_search(X, y, gpu_available, ticker, ticker_df, csv_file_path):
    def objective(trial):
        param = {
            'verbosity': 0,
            'objective': 'binary:logistic',
            'eval_metric': 'logloss',
            'tree_method': 'hist',
            'device': 'cuda' if gpu_available else 'cpu',  
            'use_label_encoder': False,
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'gamma': trial.suggest_float('gamma', 0, 5),
            'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),  # Adjusting range
            'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),     # Adjusting range
            'early_stopping_rounds': 50 
        }

        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
        model = XGBClassifier(**param)
        model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)
        preds = model.predict(X_valid)
        accuracy = accuracy_score(y_valid, preds)
        return accuracy

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=100)

    best_model = XGBClassifier(**study.best_params)
    best_model.fit(X, y, eval_set=[(X, y)], verbose=False)

    model_path = f'../models/xgboost/xbclassifier/{ticker}_best_model.pkl'

    # Update ticker_df
    if ticker in ticker_df['Ticker_Symbol'].values:
        current_score = ticker_df.loc[ticker_df['Ticker_Symbol'] == ticker, 'Best_XGBClassifier_Classification_Accuracy'].values[0]
        if pd.isnull(current_score) or study.best_value > current_score:
            ticker_df.loc[ticker_df['Ticker_Symbol'] == ticker, ['Best_XGBClassifier_Classification_Accuracy', 'Best_XGBClassifier_Classification_Path']] = [study.best_value, model_path]
            joblib.dump(best_model, model_path)
            ticker_df.to_csv(csv_file_path, index=False)
            print(f"Best model for {ticker} saved with accuracy: {study.best_value}")
             # Save feature importances
            feature_importances = pd.DataFrame(best_model.feature_importances_, index=X.columns, columns=['importance'])
            feature_importances = feature_importances.sort_values(by='importance', ascending=False)
            feature_importances.to_csv(f'../feature-importances/xbclassifier/{ticker}_feature_importances.csv')
            print(f"Feature importances for {ticker} saved.")
        else:
            print(f"Previous model accuracy: {current_score} is better for {ticker} than accuracy: {study.best_value}")
    else:
        new_row = pd.DataFrame({'Ticker_Symbol': [ticker], 'Best_XGBClassifier_Classification_Accuracy': [study.best_value], 'Best_XGBClassifier_Classification_Path': [model_path]})
        ticker_df = pd.concat([ticker_df, new_row], ignore_index=True)
        joblib.dump(best_model, model_path)
        ticker_df.to_csv(csv_file_path, index=False)
        print(f"Best model for {ticker} saved with accuracy: {study.best_value}")

        # Save feature importances
        feature_importances = pd.DataFrame(best_model.feature_importances_, index=X.columns, columns=['importance'])
        feature_importances = feature_importances.sort_values(by='importance', ascending=False)
        feature_importances.to_csv(f'../feature-importances/xbclassifier/{ticker}_feature_importances.csv')
        print(f"Feature importances for {ticker} saved.")

    return ticker_df


def xbregressor_hyperparameters_search(X, y, gpu_available, ticker, ticker_df, csv_file_path):
    def objective(trial):
        param = {
            'verbosity': 0,
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'tree_method': 'hist',
            'device': 'cuda' if gpu_available else 'cpu',  
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'gamma': trial.suggest_float('gamma', 0, 5),
            'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),  # Adjusting range
            'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),     # Adjusting range
            'early_stopping_rounds': 50 
        }

        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
        model = XGBRegressor(**param)
        model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)
        preds = model.predict(X_valid)
        rmse = root_mean_squared_error (y_valid, preds)
        return rmse

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)

    best_model = XGBRegressor(**study.best_params)
    best_model.fit(X, y, eval_set=[(X, y)], verbose=False)

    model_path = f'../models/xgboost/xbclassifier/{ticker}_best_model.pkl'
    # Update ticker_df
    if ticker in ticker_df['Ticker_Symbol'].values:
        current_score = ticker_df.loc[ticker_df['Ticker_Symbol'] == ticker, 'Best_XGBRegressor_Regression_RMSE'].values[0]
        if pd.isnull(current_score) or study.best_value < current_score:
            ticker_df.loc[ticker_df['Ticker_Symbol'] == ticker, ['Best_XGBRegressor_Regression_RMSE', 'Best_XGBRegressor_Regression_Path']] = [study.best_value, model_path]
            joblib.dump(best_model, model_path)
            ticker_df.to_csv(csv_file_path, index=False)
            print(f"Best model for {ticker} saved with RMSE: {study.best_value}")

            # Save feature importances
            feature_importances = pd.DataFrame(best_model.feature_importances_, index=X.columns, columns=['importance'])
            feature_importances = feature_importances.sort_values(by='importance', ascending=False)
            feature_importances.to_csv(f'../feature-importances/xbregressor/{ticker}_feature_importances.csv')
            print(f"Feature importances for {ticker} saved.")
        else:
            print(f"Previous model MSE: {current_score} is better for {ticker} than MSE: {study.best_value}")
    else:
        new_row = pd.DataFrame({'Ticker_Symbol': [ticker], 'Best_XGBRegressor_Regression_RMSE': [study.best_value], 'Best_XGBRegressor_Regression_Path': [model_path]})
        ticker_df = pd.concat([ticker_df, new_row], ignore_index=True)
        joblib.dump(best_model, model_path)
        ticker_df.to_csv(csv_file_path, index=False)
        print(f"Best model for {ticker} saved with RMSE: {study.best_value}")
        # Save feature importances
        feature_importances = pd.DataFrame(best_model.feature_importances_, index=X.columns, columns=['importance'])
        feature_importances = feature_importances.sort_values(by='importance', ascending=False)
        feature_importances.to_csv(f'../feature-importances/xbregressor/{ticker}_feature_importances.csv')
        print(f"Feature importances for {ticker} saved.")
   
    
    return ticker_df

In [6]:
for ticker in ticker_list:
    dataframe = pd.read_csv(f"../data/train/{ticker}.csv")
    X, y_classifier, y_regressor = preprocess_data(dataframe)
    ticker_df = xbclassifier_hyperparameters_search(X, y_classifier, gpu_available, ticker, ticker_df, csv_file_path)
    ticker_df = xbregressor_hyperparameters_search(X, y_regressor, gpu_available, ticker, ticker_df, csv_file_path)


[I 2024-09-08 09:21:22,252] A new study created in memory with name: no-name-0c40102f-9f90-4e9e-9f14-997319eb6c80
[I 2024-09-08 09:21:22,584] Trial 0 finished with value: 0.5774647887323944 and parameters: {'n_estimators': 380, 'max_depth': 4, 'learning_rate': 0.12055248516859886, 'subsample': 0.5774108552910715, 'colsample_bytree': 0.8492364970965298, 'gamma': 3.1358740899249944, 'lambda': 0.07245689631083534, 'alpha': 3.626913090421001e-08}. Best is trial 0 with value: 0.5774647887323944.
[I 2024-09-08 09:21:22,845] Trial 1 finished with value: 0.6056338028169014 and parameters: {'n_estimators': 644, 'max_depth': 10, 'learning_rate': 0.03820940997111251, 'subsample': 0.7402248358487904, 'colsample_bytree': 0.6163649484214372, 'gamma': 4.319102423143956, 'lambda': 0.0001762714523508724, 'alpha': 0.0004625156927669374}. Best is trial 1 with value: 0.6056338028169014.
[I 2024-09-08 09:21:23,204] Trial 2 finished with value: 0.6197183098591549 and parameters: {'n_estimators': 416, 'max_d

Best model for CL=F saved with accuracy: 0.676056338028169
Feature importances for CL=F saved.


[I 2024-09-08 09:21:57,493] Trial 0 finished with value: 1.6238789908489388 and parameters: {'n_estimators': 608, 'max_depth': 7, 'learning_rate': 0.022575167080379162, 'subsample': 0.6397141352718803, 'colsample_bytree': 0.5128198638416016, 'gamma': 3.522706514950027, 'lambda': 1.9008782845221154e-06, 'alpha': 5.753347032523974e-05}. Best is trial 0 with value: 1.6238789908489388.
[I 2024-09-08 09:21:58,033] Trial 1 finished with value: 1.6353393509503271 and parameters: {'n_estimators': 946, 'max_depth': 10, 'learning_rate': 0.09460946532682983, 'subsample': 0.5992995016456817, 'colsample_bytree': 0.6194938958211136, 'gamma': 1.0640357511652188, 'lambda': 2.808777512175372e-05, 'alpha': 1.1770750793408019e-07}. Best is trial 0 with value: 1.6238789908489388.
[I 2024-09-08 09:21:58,304] Trial 2 finished with value: 1.5684258417809718 and parameters: {'n_estimators': 333, 'max_depth': 3, 'learning_rate': 0.18699269113661693, 'subsample': 0.830040542522037, 'colsample_bytree': 0.6738979

Best model for CL=F saved with RMSE: 1.477026258386799
Feature importances for CL=F saved.


[I 2024-09-08 09:22:37,095] Trial 0 finished with value: 0.5915492957746479 and parameters: {'n_estimators': 982, 'max_depth': 3, 'learning_rate': 0.07106460583505923, 'subsample': 0.9842723343691487, 'colsample_bytree': 0.710832402773985, 'gamma': 2.5910301070998862, 'lambda': 2.4870288030618616e-08, 'alpha': 0.009108012283782982}. Best is trial 0 with value: 0.5915492957746479.
[I 2024-09-08 09:22:37,390] Trial 1 finished with value: 0.5915492957746479 and parameters: {'n_estimators': 728, 'max_depth': 7, 'learning_rate': 0.07453646581588119, 'subsample': 0.6369949924911991, 'colsample_bytree': 0.5197182311942246, 'gamma': 3.832029969777579, 'lambda': 3.1035663914999184e-06, 'alpha': 1.8386912350123377e-07}. Best is trial 0 with value: 0.5915492957746479.
[I 2024-09-08 09:22:37,672] Trial 2 finished with value: 0.49295774647887325 and parameters: {'n_estimators': 331, 'max_depth': 3, 'learning_rate': 0.2740855403226197, 'subsample': 0.6541612474830174, 'colsample_bytree': 0.921686861

...

Best model for SGDUSD=X saved with RMSE: 0.0018194272128071154
Feature importances for SGDUSD=X saved.


[I 2024-09-08 09:24:48,669] Trial 0 finished with value: 0.6216216216216216 and parameters: {'n_estimators': 110, 'max_depth': 10, 'learning_rate': 0.09408042048701355, 'subsample': 0.7397826072276069, 'colsample_bytree': 0.806135848714381, 'gamma': 1.019689984299312, 'lambda': 5.500972401583863e-06, 'alpha': 2.9446577874737826e-06}. Best is trial 0 with value: 0.6216216216216216.
[I 2024-09-08 09:24:48,937] Trial 1 finished with value: 0.7027027027027027 and parameters: {'n_estimators': 987, 'max_depth': 9, 'learning_rate': 0.15958060909109895, 'subsample': 0.5385847543035075, 'colsample_bytree': 0.8501017533004985, 'gamma': 2.94790220619471, 'lambda': 0.07852678156812222, 'alpha': 3.7661664582213332e-06}. Best is trial 1 with value: 0.7027027027027027.
[I 2024-09-08 09:24:49,196] Trial 2 finished with value: 0.581081081081081 and parameters: {'n_estimators': 664, 'max_depth': 7, 'learning_rate': 0.2277894979998564, 'subsample': 0.5070415813730835, 'colsample_bytree': 0.62476193953012

Best model for USDSGD=X saved with accuracy: 0.7162162162162162
Feature importances for USDSGD=X saved.


[I 2024-09-08 09:25:26,086] Trial 0 finished with value: 0.003282923869661229 and parameters: {'n_estimators': 968, 'max_depth': 6, 'learning_rate': 0.16891916159811646, 'subsample': 0.6451021634948133, 'colsample_bytree': 0.5836382616546231, 'gamma': 3.750316264557404, 'lambda': 4.437584054004322e-05, 'alpha': 7.77609047668503e-05}. Best is trial 0 with value: 0.003282923869661229.
[I 2024-09-08 09:25:26,296] Trial 1 finished with value: 0.0032902242480552413 and parameters: {'n_estimators': 842, 'max_depth': 7, 'learning_rate': 0.026213983972333897, 'subsample': 0.6353531363615762, 'colsample_bytree': 0.7458674839883315, 'gamma': 4.899873724430185, 'lambda': 4.253148876011356e-06, 'alpha': 0.008432964417144041}. Best is trial 0 with value: 0.003282923869661229.
[I 2024-09-08 09:25:26,631] Trial 2 finished with value: 0.003289209937008108 and parameters: {'n_estimators': 448, 'max_depth': 7, 'learning_rate': 0.07779568684854837, 'subsample': 0.9357971445262012, 'colsample_bytree': 0.9

Best model for USDSGD=X saved with RMSE: 0.0032813626290637897
Feature importances for USDSGD=X saved.
